# Extension Problem 57 from Hock-Schittkowski's collection

Classification : SQR-P1-1

Number of variables : n = 2

Number of constraints : 3 (1 inequality + 2 bounds)

# Modèle

Let $m \in \mathbb{N}^*$ the number of residuals

$\left\{  
\begin{array}{l l l l} 
\min f(x) \\ 
\text{s.c.}\\
0.49x_2-x_1x_2-0.09 &\geq 0\\
x_1\geq 0.4,\  x_2 \geq -4
\end{array} \right.$

with $f : x \longmapsto \sum\limits_{i=1}^{m} f_i(x)^2$

and $f_i(x) = b_i - x_1 - (0.49-x_1)\exp(-x_2(a_i-8)) \text{ for }i=1,\ldots,m$ 

Departure point : $x_0 = (0.42,5)$

Expected solution : $x^* = (0.419952675,1.284845629)$

Objective function at the solution : $f(x^*) =0.02845966972$

In [1]:
include("../src/enlsip_functions.jl")

enlsip

In [54]:
n = 2
m = 5000
nb_eq = 0 # nombre de contraintes d'égalité
nb_constraints = 3 # nombre de contraintes d'égalité et d'inégalité

3

In [4]:
# Model with expected parameters

h = x -> 0.419952675 + (0.49-0.419952675) * exp(-1.284845629*(x-8))

#36 (generic function with 1 method)

### Données d'entrée

In [55]:
# Data set from the original problem

#a = [8.,8.,10.,10.,10.,10.,12.,12.,12.,12.,14.,14.,14.,16.,16.,16.,18.,18.,20.,20.,20.,22.,22.,22., 
#         24.,24.,24.,26.,26.,26.,28.,28.,30.,30.,30.,32.,32.,34.,36.,36.,38.,38.,40.,42.]

#b = [.49,.49,.48,.47,.48,.47,.46,.46,.45,.43,.45,.43,.43,.44,.43,.43,.46,.45,.42,.42,.43,.41,
#         .41,.40,.42,.40,.40,.41,.40,.41,.41,.40,.40,.40,.38,.41,.40,.40,.41,.38,.40,.40,.39,.39]

# Entry data randomly generated between 8 and 42
# Output generated using model with expected parameters with gaussian noise

noise = 5e-2

a = range(8, 42, length = m)
b = h.(a) + noise*randn(m)
 

5000-element Vector{Float64}:
 0.4017098731177634
 0.5514144898935941
 0.5212816829467923
 0.5429195193867788
 0.578477991784928
 0.48786018536468584
 0.5279240075671945
 0.48853892154258527
 0.44378062751668385
 0.5276613306736758
 0.4926105775700036
 0.40533859568954367
 0.48448894726895997
 ⋮
 0.43203928940210923
 0.37181107548832415
 0.3405222577671956
 0.3532825812529093
 0.4692515187565545
 0.45359860786516837
 0.39228106808901186
 0.4416794595432301
 0.39513575199452505
 0.4805305329827037
 0.4584557518911522
 0.45422477232959785

In [56]:
# Résidus et matrice jacobienne associée

r_i(x::Vector,t::Float64) = x[1] + (0.49 - x[1]) * exp(-x[2]*(t - 8))
res57 = ResidualsEval(0)

function (res57::ResidualsEval)(x::Vector,rx::Vector,J::Matrix)
    # Evaluate the residuals
    if abs(res57.ctrl) == 1
        rx[:] = b - (t::Float64 -> r_i(x,t)).(a)

    # The jacobian is computed numericaly using forward differences
    # ctrl is set to 0
    elseif res57.ctrl == 2 res57.ctrl = 0 end
    return
end

In [57]:
# Contraintes et matrice jacobienne associée

cons57 = ConstraintsEval(0)

function (cons57::ConstraintsEval)(x::Vector,cx::Vector,A::Matrix)
    # Evaluate the constraints
    if abs(cons57.ctrl) == 1
        cx[:] = [0.49 * x[2] - x[1] * x[2] - 0.09, x[1] - 0.4, x[2] + 4]
    
    # The jacobian is computed anaticaly
    elseif cons57.ctrl == 2
        A[:] = [-x[2] 0.49-x[1];
        1.0 0.0;
        0.0 1.0]
    end
end

In [58]:
# Relative precision

ε = eps(Float64)
sqrt_ε = sqrt(eps(Float64))

1.4901161193847656e-8

# Resolve zith ENLSIP-0.4.0

In [59]:
x0 = [0.42,5.0]
enlsip57 = enlsip(x0,res57,cons57,n,m,nb_eq,nb_constraints,ε_abs = ε, ε_rel = sqrt_ε, ε_x = sqrt_ε, ε_c = sqrt_ε)


****************************************
*                                      *
*          ENLSIP-JULIA-0.4.0          *
*                                      *
****************************************

Starting point : [0.42, 5.0]

Number of equality constraints   : 0
Number of inequality constraints : 3
Constraints internal scaling     : false

iter    objective    cx_sum   reduction     ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  1.262391e+01  0.00e+00   1.12e-01   6.073e+00   0     2   6.04e-01    0.00e+00     1.00e-01     -
   1  1.262359e+01  0.00e+00   3.22e-04   4.795e-02   0     2   8.15e-01    7.20e-02     1.00e-01     -
   2  1.262358e+01  5.28e-11   6.96e-06   6.197e-04   1     1   1.00e+00    1.25e-01     1.00e-01    (1)
   3  1.262358e+01  5.76e-16   4.44e-10   2.034e-07   1     1   1.00e+00    3.27e-04     1.54e+06    (1)

Exit code = 10040
Number of iterations = 4 

Terminated at point : 4.204464e-01  1.293966e+00 

Active constraints 

ENLSIP([0.420446412990481, 1.2939663340106904], 12.623579146004476)

In [60]:
x1_sol = enlsip57.sol[1]
x2_sol = enlsip57.sol[2]

println("Solution found with ENLSIP :")
@printf "x_sol = (%.8e, %.9e)\n" x1_sol x2_sol
@printf "f(x_sol) = %.9e\n" enlsip57.obj_value  

println("\nOriginal parameters :")
@printf "x_opt = (%.8e, %.9e)\n" 0.419952675 1.284845629
@printf "f(x_opt) = %.9e" 0.02845966972

Solution found with ENLSIP :
x_sol = (4.20446413e-01, 1.293966334e+00)
f(x_sol) = 1.262357915e+01

Original parameters :
x_opt = (4.19952675e-01, 1.284845629e+00)
f(x_opt) = 2.845966972e-02

# Resolve using Ipopt

In [23]:
using JuMP, Ipopt

In [37]:
model = Model(with_optimizer(Ipopt.Optimizer))
@variable(model, x1, start = 0.42)
@variable(model, x2, start = 5.)

function f_i(x1, x2, t::Float64)
    return x1 + (0.49 - x1) * exp(-x2*(t - 8))
end

function f(x1, x2)
    y = b - (t::Float64 -> f_i(x1,x2,t)).(a)
    return dot(y,y) 
end

JuMP.register(model, :f, 2, f, autodiff=true)

@NLconstraint(model, c1, 0.49*x2 - x1*x2 - 0.09 >= 0)
@constraint(model, x1 >= 0.4)
@constraint(model, x2 >= -4)

@NLobjective(model, Min, f(x1,x2))

JuMP.optimize!(model)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.2477566e+00 0.00e+00 2.88e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00  

In [38]:
println("Solution ENLSIP-Julia :")
@printf "x_sol = (%.8e, %.9e)\n" x1_sol x2_sol
@printf "f(x_sol) = %.9e\n" enlsip57.obj_value 
println("\nSolution Ipopt :")
@printf "x_opt = (%.8e, %.9e)\n" JuMP.value(x1) JuMP.value(x2)
@printf "f(x_opt) = %.9e " JuMP.objective_value(model)

Solution ENLSIP-Julia :
x_sol = (4.19899806e-01, 1.283876610e+00)
f(x_sol) = 1.003088726e+00

Solution Ipopt :
x_opt = (4.19899805e-01, 1.283876467e+00)
f(x_opt) = 1.003088725e+00 